# Advent of Code 2018
We have to save christmas by solving 50 puzzles and going back in time.

First I define some utility functions, e.g. to retrieve the input file for a given day using the sessioncookie stored in the file `cookie`.

In [1]:
import requests
import os
def Input(day):
    if not os.path.exists(f'day{day}.input'):
        print(f'downloading input file for day {day}')
        with open('cookie') as f:
            cookie = f.read().strip()
        resp = requests.get(f'https://adventofcode.com/2018/day/{day}/input',
                            cookies={'session': cookie})
        if resp.status_code == 200:
            with open(f'day{day}.input', 'w') as f:
                f.write(resp.text)
        else:
            print(f'Coockie might be expired: {res.status_code} {res.reason}')
    with open(f'day{day}.input') as f:
        # discard the last line, since it is empty
        return f.read().split('\n')[:-1]

## Day 1
We have a frequency-changing device, so first we look at the end-frequency after all changes in the file are applied to the start-frequency of 0. The second question is, which frequency is reached twice first, possibly having to loop the input multiple times

In [2]:
def end_frequency(input):
    return sum([ int(l) for l in input])
assert end_frequency(['+1']) == 1
assert end_frequency([-1, 2,-2,5]) == 4
end_frequency(Input(1))

490

In [3]:
def find_duplicate_frequency(changes):
    reached_frequencies = set([0])
    current_frequency = 0
    for i in range(10**100):
        for l in changes:
            current_frequency += int(l)
            if current_frequency in reached_frequencies:
                #print(f'iteration: {i}, {current_frequency}')
                return current_frequency
            reached_frequencies.add(current_frequency)
assert find_duplicate_frequency([1,-1]) == 0
assert find_duplicate_frequency([4, -2]) == 4
find_duplicate_frequency(Input(1))

70357

## Day 2
The special fabric for Santa's new suit has gone missing. After sneaking into the storageroom, I scan the boxes and obtain a list of boxids. First, I have to make sure, every box has been scanned by computing the checksum over all boxids by multiplying the number of ids containing the same character 2 and 3 times respectively.

In [4]:
from collections import Counter
def box_checksum(boxids):
    triples = 0
    doubles = 0
    counters = [Counter(bid) for bid in boxids]
    doubles, triples = sum([1 for c in counters if 2 in c.values()]), sum([1 for c in counters if 3 in c.values()])
    return doubles * triples

assert box_checksum(['aa']) == 0
assert box_checksum(['aa', 'abbaa']) == 2
assert box_checksum(['abc', 'aaaab', 'abab', 'abcabb']) == 2
box_checksum(Input(2))

4980

So we scanned all the boxes, so now I can search for the right 2 boxes. They where manifactured very close to each other, meaning their ids are almost identical. Only the char at one position is different. So we have to return the remainder of the id.

In [5]:
import itertools
def common_chars_in_boxids(boxids):
    for s in itertools.product(boxids,repeat=2):
        matches = [ pair[0] for pair in zip(s[0],s[1]) if pair[0] == pair[1]]
        if len(s[0])-1 == len(matches):
            #print(s,matches)
            return "".join(matches)

common_chars_in_boxids(['abc','cde','bbc']) == 'bc'
common_chars_in_boxids(['axc','cde','abc']) == 'ac'
common_chars_in_boxids(Input(2))

'qysdtrkloagnfozuwujmhrbvx'

## Day 3
While the elves have found the fabric, they can't decide how to cut it. They each want to cut a rectangle out of it, specified like `#<ID> @ <off_left>,<off_right>: <width>x<height>`. As a helping hand, I have to determine how many squares are claimed by more than one rectangle.

We mark each square of the sheet with the id of each rectangle containing it. Now we only have to count all squares having more than one mark.

In [6]:
from collections import defaultdict
import re

test_claims = ["#1 @ 1,3: 4x4","#2 @ 3,1: 4x4","#3 @ 5,5: 2x2"]

def get_claims(lines):
    """ Yields (id, off_left, off_right, width, height)"""
    for l in lines:
        rv = tuple(re.findall('\\d+',l))
        assert len(rv) == 5
        yield rv
assert list(get_claims(test_claims))[0] == ('1','1','3','4','4')

def mark_sheet(claims):
    marks = defaultdict(set)
    for c in claims:
        for x in range(int(c[1]), int(c[1])+int(c[3])):
            for y in range(int(c[2]), int(c[2])+int(c[4])):
                marks[(x,y)].add(c[0])
    return marks

def count_multi_marks(claims):
    return len([1 for x in mark_sheet(claims).values() if len(x) > 1])

assert count_multi_marks([(0,2,2,4,4), (1,1,1,1,1)]) == 0
assert count_multi_marks([(0,2,2,4,3), (1,1,1,2,2)]) == 1
assert count_multi_marks(get_claims(test_claims)) == 4
assert count_multi_marks([(0,2,2,4,3), (1,1,1,2,2), (2,2,2,1,1)]) == 1

count_multi_marks(get_claims(Input(3)))

118322

There seems to be one claim, that has no intersections with any of the others. If I find it, perhaps the elves can actually work from there.

In [7]:
def find_best_claim(claims):
    all_ids = set()
    intersecting_ids = set()
    for v in mark_sheet(claims).values():
        if len(v) > 1:
            intersecting_ids.update(v)
            #print(intersecting_ids)
        all_ids.update(v)
    # there has to be one
    return all_ids.difference(intersecting_ids).pop()
assert find_best_claim(get_claims(test_claims)) == '3'
find_best_claim(get_claims(Input(3)))

'1178'